# **Descripción del Notebook**

Este notebook tiene como objetivo **manipular datos a partir de un dataset original, realizar transformaciones y, finalmente, exportar los datos procesados a MySQL Workbench**. A continuación, se detalla la estructura del notebook y los pasos principales que se siguen.

## **Estructura del Notebook**

1. **Importación de Librerías**
   En esta primera sección, se importan las librerías necesarias para la manipulación de datos y la conexión con la base de datos MySQL. Entre las librerías utilizadas destacan:

   - `pandas`: Para la manipulación y transformación de datos.
   - `pymysql`: Para interactuar con la base de datos MySQL Workbench.

2. **Carga de Datos**
   Se procede a la carga de los datos desde el archivo fuente. Estos datos serán procesados en las siguientes secciones.

3. **Creación de DataFrames Secundarios**
   A partir del DataFrame original, se crean varios **DataFrames más pequeños** mediante filtros y selecciones específicas. Además, se llevan a cabo las siguientes transformaciones:
   
   - **Creación de nuevas columnas**: Se generan columnas adicionales basadas en cálculos o transformaciones de las columnas existentes.
   - **Renombramiento de columnas**: Para facilitar el análisis y claridad del código, se renombrarán algunas columnas.

4. **Exportación de los DataFrames a MySQL Workbench**
   En esta sección, se realiza la conexión a MySQL Workbench utilizando la librería `pymysql` y se exportan los DataFrames previamente creados a la base de datos. Cada DataFrame se convierte en una tabla dentro de MySQL Workbench, respetando los nombres y formatos establecidos.

## **Conclusión**
Al final de este notebook, los datos han sido **procesados, transformados y almacenados** en MySQL, listos para futuras consultas o análisis. Este flujo permite una interacción ágil entre el análisis de datos en Python y el almacenamiento relacional en MySQL.


In [1]:
import pandas as pd
from functions import *

url = "df_limpio.csv"
df = read_data(url)

In [2]:
df = clean_original_data(df)
df

,sex,age,level_education,country,year,value_unemployment,value_education,valor_educacion,indice_educativo
0,Females,15-29,ESO,Austria,2014,15.7,27.9,1,2.84
1,Females,15-29,ESO,Austria,2015,15.3,27.5,1,2.89
2,Females,15-29,ESO,Austria,2016,17.1,27.3,1,2.91
3,Females,15-29,ESO,Austria,2017,16.8,27.2,1,2.92
4,Females,15-29,ESO,Austria,2018,13.4,27.1,1,2.92
...,...,...,...,...,...,...,...,...,...
4636,Males,15-29,UNIVERSIDAD,United Kingdom,2015,6.2,26.6,5,3.14
4637,Males,15-29,UNIVERSIDAD,United Kingdom,2016,5.3,27.1,5,3.14
4638,Males,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,27.1,5,3.14
4639,Males,15-29,UNIVERSIDAD,United Kingdom,2018,5.0,28.1,5,3.17


### **Creación del DataFrame `country`**

En esta sección, se genera un **nuevo DataFrame llamado `country`** que contiene dos columnas:

1. **`id_country`**: Un identificador para los países basado en códigos de dos letras (ISO).
2. **`country_name`**: El nombre de los países extraído de la columna `country` en el DataFrame original.

La función `new_df_country(df)` toma como entrada el DataFrame original, selecciona los nombres únicos de los países y crea un DataFrame nuevo con los identificadores y nombres de los países.

El nuevo DataFrame `country` se utilizará posteriormente para realizar transformaciones y cargar la información en la base de datos.


In [3]:
country = new_df_country(df)
country

,id_country,country_name
0,AT,Austria
1,BE,Belgium
2,BG,Bulgaria
3,CH,Croatia
4,CY,Cyprus
5,CZ,Czechia
6,DE,Denmark
7,DK,Estonia
8,EE,Finland
9,EL,France


### **Creación del DataFrame `gender`**

En esta sección, se crea un **nuevo DataFrame llamado `gender`** que contiene dos columnas:

1. **`id_gender`**: Un identificador de género que utiliza las letras "F" (Female) y "M" (Male).
2. **`gender`**: Los valores únicos de la columna `sex` extraídos del DataFrame original.

La función `new_df_gender(df)` toma el DataFrame original y extrae los géneros únicos, para luego asociarlos con identificadores predefinidos. Este DataFrame se utilizará más adelante para normalizar y almacenar la información de género en la base de datos.


In [4]:
gender = new_df_gender(df)
gender

,id_gender,gender
0,F,Females
1,M,Males


### **Creación del DataFrame `level_education`**

En esta sección, se crea un **nuevo DataFrame llamado `level_education`** que contiene dos columnas:

1. **`id_level`**: Un identificador para los niveles educativos que incluye:
   - `ESO`: Educación Secundaria Obligatoria
   - `BACH-FP`: Bachillerato y Formación Profesional
   - `BACH`: Bachillerato
   - `FP`: Formación Profesional
   - `UNIVERSIDAD`: Educación Universitaria

2. **`valor_educacion`**: Los valores únicos de la columna `valor_educacion` del DataFrame original.

La función `new_df_level(df)` toma el DataFrame original y extrae los niveles educativos únicos para luego asociarlos con los identificadores predefinidos. Este DataFrame será utilizado para normalizar y almacenar la información sobre niveles educativos en la base de datos.


In [5]:
level_education = new_df_level(df)
level_education

,id_level,valor_educacion
0,ESO,1
1,BACH-FP,3
2,BACH,4
3,FP,2
4,UNIVERSIDAD,5


### **Conversión de Género a Identificadores**

En esta sección, se realiza la conversión de los valores de género en la columna `sex` del DataFrame original a un formato estándar de identificador de género:

1. **Conversión de Valores**: Se usa la función `convert_gender` para transformar los valores en la columna `sex`:
   - `Females` se convierte en `F`.
   - `Males` se convierte en `M`.
   - Otros valores se marcan como `Unknown` para manejar casos no especificados.

2. **Aplicación de la Función**: La función `convert_gender` se aplica a cada valor de la columna `sex` para crear una nueva columna `id_gender` con los identificadores correspondientes.

Esto asegura que los valores de género en el DataFrame estén en un formato estandarizado y listo para ser utilizado en análisis posteriores o almacenamiento en la base de datos.


In [7]:

df['id_gender'] = df['sex'].apply(convert_gender)
df

,sex,age,level_education,country,year,value_unemployment,value_education,valor_educacion,indice_educativo,id_gender
0,Females,15-29,ESO,Austria,2014,15.7,27.9,1,2.84,F
1,Females,15-29,ESO,Austria,2015,15.3,27.5,1,2.89,F
2,Females,15-29,ESO,Austria,2016,17.1,27.3,1,2.91,F
3,Females,15-29,ESO,Austria,2017,16.8,27.2,1,2.92,F
4,Females,15-29,ESO,Austria,2018,13.4,27.1,1,2.92,F
...,...,...,...,...,...,...,...,...,...,...
4636,Males,15-29,UNIVERSIDAD,United Kingdom,2015,6.2,26.6,5,3.14,M
4637,Males,15-29,UNIVERSIDAD,United Kingdom,2016,5.3,27.1,5,3.14,M
4638,Males,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,27.1,5,3.14,M
4639,Males,15-29,UNIVERSIDAD,United Kingdom,2018,5.0,28.1,5,3.17,M


### **Creación de la Columna `id_country`**

En esta sección, se crea una nueva columna en el DataFrame que convierte los nombres de los países a códigos de país ISO de dos letras. Esto se realiza mediante la función `colum_id_map`.

1. **Mapa de Países**: La función utiliza un diccionario `country_map` que mapea los nombres completos de los países a sus respectivos códigos ISO.
2. **Aplicación del Mapa**: La función `map` se usa para transformar los valores en la columna `country` del DataFrame a sus códigos correspondientes y almacenar estos códigos en una nueva columna llamada `id_country`.

Esto facilita la estandarización de los datos de país y permite su integración más sencilla con bases de datos u otros sistemas que utilicen códigos de país.


In [8]:
df = colum_id_map(df)
df

,sex,age,level_education,country,year,value_unemployment,value_education,valor_educacion,indice_educativo,id_gender,id_country
0,Females,15-29,ESO,Austria,2014,15.7,27.9,1,2.84,F,AT
1,Females,15-29,ESO,Austria,2015,15.3,27.5,1,2.89,F,AT
2,Females,15-29,ESO,Austria,2016,17.1,27.3,1,2.91,F,AT
3,Females,15-29,ESO,Austria,2017,16.8,27.2,1,2.92,F,AT
4,Females,15-29,ESO,Austria,2018,13.4,27.1,1,2.92,F,AT
...,...,...,...,...,...,...,...,...,...,...,...
4636,Males,15-29,UNIVERSIDAD,United Kingdom,2015,6.2,26.6,5,3.14,M,UK
4637,Males,15-29,UNIVERSIDAD,United Kingdom,2016,5.3,27.1,5,3.14,M,UK
4638,Males,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,27.1,5,3.14,M,UK
4639,Males,15-29,UNIVERSIDAD,United Kingdom,2018,5.0,28.1,5,3.17,M,UK


### **Renombrar y Crear Columnas en el DataFrame**

En esta sección, se realizan dos modificaciones importantes en el DataFrame:

1. **Renombramiento de Columna**: La columna `level_education` se renombra a `id_level` para estandarizar los nombres de las columnas según las convenciones de la base de datos o los requisitos del análisis.
2. **Creación de Nueva Columna**: Se añade una nueva columna llamada `id_datos` que contiene un identificador único para cada fila, que va del 1 al número total de filas en el DataFrame.

Estas modificaciones preparan el DataFrame para la integración con otros sistemas o bases de datos, facilitando el análisis y la organización de los datos.


In [9]:
df = rename_column_and_new_column(df)
df

,sex,age,id_level,country,year,value_unemployment,value_education,valor_educacion,indice_educativo,id_gender,id_country,id_datos
0,Females,15-29,ESO,Austria,2014,15.7,27.9,1,2.84,F,AT,1
1,Females,15-29,ESO,Austria,2015,15.3,27.5,1,2.89,F,AT,2
2,Females,15-29,ESO,Austria,2016,17.1,27.3,1,2.91,F,AT,3
3,Females,15-29,ESO,Austria,2017,16.8,27.2,1,2.92,F,AT,4
4,Females,15-29,ESO,Austria,2018,13.4,27.1,1,2.92,F,AT,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4636,Males,15-29,UNIVERSIDAD,United Kingdom,2015,6.2,26.6,5,3.14,M,UK,3110
4637,Males,15-29,UNIVERSIDAD,United Kingdom,2016,5.3,27.1,5,3.14,M,UK,3111
4638,Males,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,27.1,5,3.14,M,UK,3112
4639,Males,15-29,UNIVERSIDAD,United Kingdom,2018,5.0,28.1,5,3.17,M,UK,3113


### **Creación del DataFrame `datos_edu`**

En esta sección, se crea un **nuevo DataFrame llamado `datos_edu`** que selecciona un subconjunto específico de columnas del DataFrame original. Este nuevo DataFrame contiene las siguientes columnas:

1. **`id_datos`**: Identificador único para cada fila.
2. **`indice_educativo`**: Índice relacionado con la educación.
3. **`value_unemployment`**: Valor relacionado con el desempleo.
4. **`value_education`**: Valor relacionado con la educación.
5. **`year`**: Año del registro.
6. **`id_country`**: Código del país.
7. **`id_gender`**: Identificador del género.
8. **`id_level`**: Nivel educativo.

La función `new_df_datos(df)` selecciona estas columnas para organizar los datos en un formato adecuado para análisis o exportación, simplificando el DataFrame al incluir solo las columnas relevantes.


In [10]:
datos_edu = new_df_datos(df)
datos_edu

,id_datos,indice_educativo,value_unemployment,value_education,year,id_country,id_gender,id_level
0,1,2.84,15.7,27.9,2014,AT,F,ESO
1,2,2.89,15.3,27.5,2015,AT,F,ESO
2,3,2.91,17.1,27.3,2016,AT,F,ESO
3,4,2.92,16.8,27.2,2017,AT,F,ESO
4,5,2.92,13.4,27.1,2018,AT,F,ESO
...,...,...,...,...,...,...,...,...
4636,3110,3.14,6.2,26.6,2015,UK,M,UNIVERSIDAD
4637,3111,3.14,5.3,27.1,2016,UK,M,UNIVERSIDAD
4638,3112,3.14,4.6,27.1,2017,UK,M,UNIVERSIDAD
4639,3113,3.17,5.0,28.1,2018,UK,M,UNIVERSIDAD


### **Configuración para la Exportación a MySQL**

A partir de esta sección, se configura la conexión y el proceso para exportar los datos a una base de datos MySQL.

1. **Importación de Librerías**:
   - **`create_engine` y `text` de `sqlalchemy`**: Utilizados para crear una conexión a la base de datos y ejecutar consultas SQL.
   - **`pymysql`**: Adaptador para conectar Python con MySQL, necesario para la operación con `sqlalchemy`.

Estas librerías permiten establecer una conexión con MySQL y realizar operaciones de escritura y consulta en la base de datos. A continuación, se detallará cómo se configuran y utilizan estas herramientas para la exportación de datos.


In [14]:
from sqlalchemy import create_engine, text

In [13]:
import pymysql

### **Conexión y Exportación de DataFrames a MySQL**

En esta sección, se configura la conexión a la base de datos MySQL y se exportan los DataFrames creados a la base de datos.

1. **Configuración de la Conexión**:
   - Se define el nombre de la base de datos (`bd`) y se establece una cadena de conexión (`connection_string`) que incluye el nombre de usuario, la contraseña y el nombre de la base de datos.
   - Se crea un motor de conexión (`engine`) utilizando `create_engine` de SQLAlchemy con la cadena de conexión.

2. **Exportación de DataFrames**:
   - Se utiliza el método `to_sql` de Pandas para enviar cada DataFrame a la base de datos MySQL.
   - En este ejemplo, se muestra cómo exportar el DataFrame `country` a una tabla llamada `country`. Este proceso se repetirá para los demás DataFrames (`gender`, `level_education`, `datos_edu`, etc.), cambiando el nombre de la tabla según corresponda.

   **Nota**: Asegúrate de reemplazar la contraseña en `password` con tu contraseña real antes de ejecutar el código.

Esta configuración permite almacenar los DataFrames en una base de datos MySQL para su posterior análisis o integración con otros sistemas.


In [19]:
# Tus parámetros de conexión
bd = "project_ue"
password = "cambia_contraseña"  # Asegúrate de reemplazar esto con tu contraseña real
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Enviar DataFrame a MySQL
table_name = 'country'
country.to_sql(table_name, con=engine, if_exists='append', index=False)

32

In [20]:
bd = "project_ue"
password = "cambia_contraseña"  # Asegúrate de reemplazar esto con tu contraseña real
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Enviar DataFrame a MySQL
table_name = 'gender'
gender.to_sql(table_name, con=engine, if_exists='append', index=False)

2

In [21]:
bd = "project_ue"
password = "cambia_contraseña"  # Asegúrate de reemplazar esto con tu contraseña real
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Enviar DataFrame a MySQL
table_name = 'datos_educativos'
datos_edu.to_sql(table_name, con=engine, if_exists='append', index=False)

3114

In [22]:
bd = "project_ue"
password = "cambia_contraseña"  # Asegúrate de reemplazar esto con tu contraseña real
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Enviar DataFrame a MySQL
table_name = 'level_education'
level_education.to_sql(table_name, con=engine, if_exists='append', index=False)

5